In [ ]:
## Simply placing a £2 bet to see if bet placement works thorugh API

In [1]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import git
import datetime

In [7]:
# loading in api login credentials
repo = git.Repo('.', search_parent_directories=True)
logins_dir = str(repo.working_tree_dir) + "/api_logins.txt"

with open(logins_dir) as f:
    login_dict =  json.load(f)

In [8]:
trading = betfairlightweight.APIClient(username=login_dict['my_username'],
                                       password=login_dict['my_password'],
                                       app_key=login_dict['my_app_key'],
                                       certs=login_dict['certs_path'])

In [9]:
trading.login()

<LoginResource>

In [49]:
# make event type request to find horse racing event type

horse_racing_event_type_id = trading.betting.list_event_types(
    filter=filters.market_filter(
        text_query='Horse Racing'
    )
)

for event_type in horse_racing_event_type_id:
    print(event_type.event_type.id, event_type.event_type.name, event_type.market_count) # prints id, name and market count
    horse_racing_id = event_type.event_type.id

    # list all horse racing market catalogues
    market_catalogues = trading.betting.list_market_catalogue(
        filter=filters.market_filter(
            event_type_ids=[horse_racing_id],  # filter on just horse racing
            market_countries=['GB'],  # filter on just GB countries
            market_type_codes=['WIN'],  # filter on just WIN market types
        ),
        
        market_projection=['MARKET_START_TIME', 'RUNNER_DESCRIPTION'],  # runner description required
        max_results=100
    )

    print('%s market catalogues returned' % len(market_catalogues))

7 Horse Racing 430
15 market catalogues returned


In [50]:
def process_runner_books(runner_books):
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]

    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [51]:
market_IDs = []
market_start_times = []
market_names = []
runner_dict = {}

for market_catalogue in market_catalogues:
    market_IDs = market_IDs + [market_catalogue.market_id]
    market_names = market_names + [market_catalogue.market_name]
    market_start_times.append(market_catalogue.market_start_time)
    
    for runner in market_catalogue.runners:
        runner_dict[runner.selection_id] = runner.runner_name
    
GB_runners_df = pd.DataFrame()  # create an empty dataframe to append other dfs to

# market book request
for market,time, name in zip(market_IDs, market_start_times, market_names):
    
    # Create a price filter. Get all traded and offer data
    price_filter = filters.price_projection(
        price_data=['EX_BEST_OFFERS']
    )
    
    market_books = trading.betting.list_market_book(
        market_ids=[market],
        price_projection=price_filter
    )
    
    # append the new market book runners to the final dataframe
    market_book = market_books[0]
    runners_df = process_runner_books(market_book.runners)
    runners_df['Event_ID'] = market
    runners_df['Event Time'] = time
    runners_df['Event Name'] = name
    GB_runners_df = GB_runners_df.append(runners_df, ignore_index=True)

GB_runners_df['Selection Name'] = GB_runners_df['Selection ID'].map(runner_dict)

In [52]:
event_dfs = []

for event_id in market_IDs:
    event_dfs.append(GB_runners_df.loc[GB_runners_df['Event_ID'] == event_id])

In [53]:
event_dfs[4]

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor,Event_ID,Event Time,Event Name,Selection Name
37,23225577,1.87,36.90,1.95,16.12,1.88,0.0,ACTIVE,None,39.815,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Skiddaw Tara
38,23109946,3.20,11.89,3.40,13.57,3.30,0.0,ACTIVE,None,41.936,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Get Help
39,27156467,8.20,10.56,38.00,37.77,13.00,0.0,ACTIVE,None,9.323,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Empire Steel
40,18765624,11.50,17.03,22.00,11.43,20.00,0.0,ACTIVE,None,5.895,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,oh No
41,22071695,21.00,13.71,390.00,12.74,85.00,0.0,ACTIVE,None,2.148,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Shoconas Joy
42,14569145,10.00,17.25,950.00,2.13,85.00,0.0,ACTIVE,None,0.441,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Sincerely Resdev
43,22350536,36.00,10.56,1000.00,2.05,250.00,0.0,ACTIVE,None,0.441,1.166347795,2019-12-15 12:50:00,3m1f Mdn Hrd,Stormy Reception


In [54]:
runners_df = event_dfs[4]

In [55]:
# looking to place a £2 back bet on highest back price
# will attempt to back runner within one minute of pulling price

In [56]:
# Get the favourite's price and selection id
fav_selection_id = runners_df.loc[runners_df['Best Back Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Back Price'].idxmin(), 'Best Back Price']
market_id = 1.166347795

In [57]:
fav_selection_id

23225577

In [58]:
# Define a limit order filter
limit_order_filter = betfairlightweight.filters.limit_order(
    size=2, 
    price=1.87,
    persistence_type='LAPSE'
)

# Define an instructions filter
instructions_filter = betfairlightweight.filters.place_instruction(
    selection_id=str(fav_selection_id),
    order_type="LIMIT",
    side="BACK",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '23225577',
 'side': 'BACK',
 'limitOrder': {'price': 1.87, 'persistenceType': 'LAPSE', 'size': 2}}

In [59]:
# placing the order
order = trading.betting.place_orders(
    market_id=market_id, # The market id we obtained from before
    customer_strategy_ref='back_the_fav',
    instructions=[instructions_filter] # This must be a list
)

In [60]:
order.__dict__

{'elapsed_time': 0.372666,
 '_datetime_created': datetime.datetime(2019, 12, 14, 23, 29, 6, 325702),
 '_datetime_updated': datetime.datetime(2019, 12, 14, 23, 29, 6, 325702),
 '_data': {'status': 'SUCCESS',
  'marketId': '1.166347795',
  'instructionReports': [{'status': 'SUCCESS',
    'instruction': {'selectionId': 23225577,
     'limitOrder': {'size': 2.0, 'price': 1.87, 'persistenceType': 'LAPSE'},
     'orderType': 'LIMIT',
     'side': 'BACK'},
    'betId': '188794499996',
    'placedDate': '2019-12-14T23:29:06.000Z',
    'averagePriceMatched': 1.87115,
    'sizeMatched': 2.0,
    'orderStatus': 'EXECUTION_COMPLETE'}]},
 'market_id': '1.166347795',
 'status': 'SUCCESS',
 'customer_ref': None,
 'error_code': None,
 'place_instruction_reports': [<betfairlightweight.resources.bettingresources.PlaceOrderInstructionReports at 0x11b73aac8>]}

In [ ]:
# success!!!

In [61]:
# dictionary of all current orders
trading.betting.list_current_orders(customer_strategy_refs=['back_the_fav']).__dict__

{'streaming_unique_id': None,
 'streaming_update': None,
 'publish_time': None,
 'elapsed_time': 0.304033,
 '_datetime_created': datetime.datetime(2019, 12, 14, 23, 30, 18, 532474),
 '_datetime_updated': datetime.datetime(2019, 12, 14, 23, 30, 18, 532474),
 '_data': {'currentOrders': [{'betId': '188794499996',
    'marketId': '1.166347795',
    'selectionId': 23225577,
    'handicap': 0.0,
    'priceSize': {'price': 1.87, 'size': 2.0},
    'bspLiability': 0.0,
    'side': 'BACK',
    'status': 'EXECUTION_COMPLETE',
    'persistenceType': 'LAPSE',
    'orderType': 'LIMIT',
    'placedDate': '2019-12-14T23:29:06.000Z',
    'matchedDate': '2019-12-14T23:29:06.000Z',
    'averagePriceMatched': 1.87115,
    'sizeMatched': 2.0,
    'sizeRemaining': 0.0,
    'sizeLapsed': 0.0,
    'sizeCancelled': 0.0,
    'sizeVoided': 0.0,
    'regulatorCode': 'GIBRALTAR REGULATOR',
    'customerStrategyRef': 'back_the_fav'}],
  'moreAvailable': False},
 'more_available': False,
 'orders': [<betfairlightwei

In [64]:
# listing cleared orders
cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
                                                    market_ids=["1.166347795"])
pd.DataFrame(cleared_orders._data['clearedOrders']) # empty, will check this once race finished

""


In [65]:
# market profit and loss
pl = trading.betting.list_market_profit_and_loss(market_ids=["1.166347795"], 
                                                 include_bsp_bets='true', 
                                                 include_settled_bets='true')
pl_df = pd.DataFrame(pl[0]._data['profitAndLosses']).assign(marketId=pl[0].market_id)
pl_df 

,ifWin,selectionId,marketId
0,-2.00,18765624,1.166347795
1,-2.00,14569145,1.166347795
2,-2.00,23109946,1.166347795
3,1.74,23225577,1.166347795
4,-2.00,27156467,1.166347795
5,-2.00,22350536,1.166347795
6,-2.00,22071695,1.166347795


In [ ]:
# this operation will still work for ~ 1min after race, so can call at endpoint of code

In [70]:
# getting account statement
# Define a date filter - get all bets for the past 4 days
four_days_ago = (datetime.datetime.utcnow() - datetime.timedelta(days=4)).strftime("%Y-%m-%dT%TZ")
acct_statement_date_filter = betfairlightweight.filters.time_range(from_=four_days_ago)

# Request account statement
account_statement = trading.account.get_account_statement(item_date_range=acct_statement_date_filter)

In [71]:
recent_transactions = pd.DataFrame(account_statement._data['accountStatement'])
recent_transactions

""


In [72]:
# Create df of itemClassData - iterate over the account statement list and convert to json so that the DataFrame function
# can read it correctly
class_data = [json.loads(account_statement.account_statement[i].item_class_data['unknownStatementItem']) 
              for i in range(len(account_statement.account_statement))]

In [73]:
class_df = pd.DataFrame(class_data)
class_df

""


In [ ]:
# unsure as to why this last part didn't work... maybe has to have settled bets first ? 